# 1- Determine the words that we will work on

In [3]:
# libraries
import os
import cv2
import mediapipe as mp
import numpy as np
import pickle
import matplotlib.pyplot as plt
import random
import tensorflow as tf

In [2]:
class_names = []
for class_name in os.listdir("arabic"):
    class_names.append(class_name.split(".")[0])
print(class_names) # 90 words

['أ', 'أب', 'أبن_أو_ولد', 'أبيض', 'أثنين', 'أحبك', 'أحمر', 'أخضر', 'أربعة', 'أسمك', 'أشارة', 'أصم', 'أنا', 'أنت', 'أين', 'الأثنين', 'الأحد', 'الأربعاء', 'الأقصر', 'الاسماعيلية', 'الثلاثاء', 'الحمد لله', 'الخميس', 'السبت', 'السلام', 'الشرقية', 'الصلاة', 'القاهرة', 'النوم_او_ينام', 'ب', 'بتشتغل', 'برتقال', 'بنت_او_ابنة', 'بنك', 'ت', 'تسعة', 'تسكن_او_ساكن', 'تيليفيزيون', 'تين', 'ث', 'ثلاثة', 'ثمانية', 'ج', 'جائع', 'جامعة', 'جيد', 'ح', 'حمام', 'خ', 'خطوبة', 'خمسة', 'د', 'دكتور', 'ذ', 'ر', 'ز', 'س', 'سبعة', 'ستة', 'سرير', 'سعيد', 'سىء', 'ش', 'ص', 'ض', 'ط', 'طاولة', 'طبق', 'طفل', 'طلاق_او_مطلق_او_مطلقة', 'ظ', 'ع', 'عشرة', 'عندك', 'غ', 'ف', 'ق', 'ك', 'كرسي', 'كلية', 'كوب', 'ل', 'م', 'ماء', 'ماذا', 'متزوج', 'مدرسة', 'مساعدة', 'مستشفى', 'مصر', 'معلم', 'منزل_او_بيت', 'مهندس', 'ن', 'نعم', 'ه', 'و', 'واحد', 'ي']


## Create folder for each class in english data directory

In [7]:
for class_name in os.listdir("arabic"):
    os.mkdir(os.path.join("arabic_data", class_name.split(".")[0]))

# 2- Collect images for each class [word] -> using web camera
- we will collect 2000 images for each class and put them in folders created in english data directory

In [2]:
# create objects just focus on the hands 
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles

# get the model that detect hand_landmarks 
hands = mp_hands.Hands(static_image_mode=True, min_detection_confidence=0.5)

In [131]:
# setup my camera [my camera has the index (0)]
my_camera = cv2.VideoCapture(0)

# define a counter to give each frame a unique name 
counter = 0

# In this loop, Frames will be taken from the camera until we stop it.
while my_camera.isOpened():
    
    # read frame by frame from the camera -> return [frame, status of the reading process (is the camera capture the frame right or not)]
    # status -> Boolean, Frame --> image with np.array data type
    status, frame = my_camera.read()
    frame = cv2.flip(frame, 1)
    frame_copy = np.copy(frame)
    
    # convert frame to rgb
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # get hands detection on the rgb image 
    result = hands.process(frame_rgb)
    
    if result.multi_hand_landmarks:
        for hand_landmark in result.multi_hand_landmarks:
            # draw the landmarks
            mp_drawing.draw_landmarks(
                frame_copy,
                hand_landmark, 
                mp_hands.HAND_CONNECTIONS,
                mp_drawing_styles.get_default_hand_landmarks_style(),
                mp_drawing_styles.get_default_hand_connections_style())
    
    # display the captured frame
    cv2.imshow('Captured Frame', frame_copy)

    # save the captured frame on pressing [s]
    # if cv2.waitKey(1) & 0xFF == ord('s'):
    #     # increase counter by 1 
    counter += 1 
    cv2.imwrite(os.path.join("arabic_data", "pray", f"new_right_pray_frame_{counter}.jpg"), frame)
    # print(f'{counter}_image successfully saved')
    
    
    # Stop the reading process on pressing [q]
    if (cv2.waitKey(1) & 0xFF == ord('q')) or counter == 2000:
        break
    
my_camera.release()
cv2.destroyAllWindows()

In [24]:
my_camera.release()
cv2.destroyAllWindows()

## Remove images that can't be detected by the google model 

In [133]:
for i in os.listdir(os.path.join("arabic_data", "pray")):
    image = cv2.imread(os.path.join("arabic_data", "pray", i))
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    result = hands.process(image_rgb)
    
    if result.multi_hand_landmarks:
        continue
    else:
        print(i)
        os.remove(os.path.join("arabic_data", "pray", i))

## Remove Frames till 2000 frame 

In [151]:
while len(os.listdir(os.path.join("english_data", "Yes"))) > 2000:
    random_image = random.sample(os.listdir(os.path.join("english_data", "Yes")), k=1)
    # print(random_image)
    random_image_path = os.path.join("english_data", "Yes", random_image[0])
    os.remove(random_image_path)

## Check that 2 hands words are detected correctly

In [1]:
for i in os.listdir(os.path.join("arabic_data", "pray")):
    image = cv2.imread(os.path.join("arabic_data", "pray", i))
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    result = hands.process(image_rgb)
    # print(i)

    if result.multi_hand_landmarks:
        extracted_frame_features = []
        for hand_landmark in result.multi_hand_landmarks:
            for landmark in hand_landmark.landmark:
                extracted_frame_features.append(landmark.x)
                extracted_frame_features.append(landmark.y)

        if len(extracted_frame_features) != 84:
            print(i)
            print(len(extracted_frame_features))
            os.remove(os.path.join("arabic_data", "pray", i))
    

## Check the balancing of the dataset

In [2]:
for class_name in os.listdir(os.path.join('arabic_data')):
    if len(os.listdir(os.path.join('arabic_data', class_name))) != 0:
        counter = 0
        for numpy_array in os.listdir(os.path.join("arabic_data", class_name)):
            if numpy_array.split(".")[-1] == "npy":
                counter += 1
        
        print(class_name)
        print(counter)

# 3- Extract Features from the images

## Process : extract features and save it in .npy file for each image

In [3]:
for class_name in os.listdir(os.path.join('arabic_data')):
    if len(os.listdir(os.path.join('arabic_data', class_name))) != 0:
        print(f"------------------------------------- {class_name} ---------------------------------")
        for image_name in os.listdir(os.path.join('arabic_data', class_name)):
            if image_name.split(".")[-1] == "jpg":
                print(image_name)
                # define the path of the image
                image_path = os.path.join('arabic_data', class_name, image_name)

                # bgr image
                image = cv2.imread(image_path)

                # rgb image 
                image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

                # process the rgb image to get the hand detection
                result = hands.process(image_rgb)

                # check if there is any detection of hands or not 
                if result.multi_hand_landmarks:

                    # define list to put in it x,y values for each landmarks 
                    current_image_landmarks = []

                    # get x and y value for each landmark
                    for hand_landmark in result.multi_hand_landmarks:
                        for landmark in hand_landmark.landmark:
                            current_image_landmarks.append(landmark.x)
                            current_image_landmarks.append(landmark.y)

                    # check that the number of landmarks are equal for each image
                    if len(current_image_landmarks) < 84:
                        current_image_landmarks = current_image_landmarks + [0]*(84-len(current_image_landmarks))

                    # save the extracted image features in numpy array 
                    save_path = os.path.join("arabic_data", class_name, f"{image_name}_features")
                    np.save(save_path, current_image_landmarks)
                    

## Create label map

In [142]:
classes = []
for class_name in os.listdir(os.path.join('arabic_data')):
    if len(os.listdir(os.path.join('arabic_data', class_name))) != 0:
        classes.append(class_name)

print(len(classes))
print(classes)

90
['aeen', 'alef', 'allah', 'bad', 'bank', 'bathroom', 'beh', 'cairo', 'chair', 'child', 'college', 'daaad', 'daal', 'deaf', 'dish', 'divorce', 'egypt', 'eight', 'engagement', 'engineer', 'father', 'feh', 'five', 'four', 'geem', 'gheen', 'girl_or_daughter', 'green', 'haa', 'happy', 'hehh', 'help', 'home_or_house', 'hospital', 'hungry', 'ismaalia', 'i_love_you', 'i_or_me', 'kaaf', 'khaa', 'laam', 'luxor', 'meem', 'monday', 'nine', 'noon', 'one', 'orange', 'pray', 'qaaf', 'raa', 'saaad', 'saturday', 'school', 'seen', 'seven', 'sharkia', 'sheen', 'sign', 'six', 'sleep_n_v', 'son_or_boy', 'tah', 'teacher', 'teen_fruit', 'teh', 'ten', 'theh', 'the_peace', 'three', 'thursday', 'tuesday', 'tv', 'two', 'university', 'waow', 'water', 'wednesday', 'what', 'where', 'white', 'work_v', 'yeeh', 'yellow', 'yes', 'you', 'your_name', 'zaal', 'zah', 'zeen']


In [143]:
class_to_index = {class_name:index for index, class_name in enumerate(classes)}
print(class_to_index)

{'aeen': 0, 'alef': 1, 'allah': 2, 'bad': 3, 'bank': 4, 'bathroom': 5, 'beh': 6, 'cairo': 7, 'chair': 8, 'child': 9, 'college': 10, 'daaad': 11, 'daal': 12, 'deaf': 13, 'dish': 14, 'divorce': 15, 'egypt': 16, 'eight': 17, 'engagement': 18, 'engineer': 19, 'father': 20, 'feh': 21, 'five': 22, 'four': 23, 'geem': 24, 'gheen': 25, 'girl_or_daughter': 26, 'green': 27, 'haa': 28, 'happy': 29, 'hehh': 30, 'help': 31, 'home_or_house': 32, 'hospital': 33, 'hungry': 34, 'ismaalia': 35, 'i_love_you': 36, 'i_or_me': 37, 'kaaf': 38, 'khaa': 39, 'laam': 40, 'luxor': 41, 'meem': 42, 'monday': 43, 'nine': 44, 'noon': 45, 'one': 46, 'orange': 47, 'pray': 48, 'qaaf': 49, 'raa': 50, 'saaad': 51, 'saturday': 52, 'school': 53, 'seen': 54, 'seven': 55, 'sharkia': 56, 'sheen': 57, 'sign': 58, 'six': 59, 'sleep_n_v': 60, 'son_or_boy': 61, 'tah': 62, 'teacher': 63, 'teen_fruit': 64, 'teh': 65, 'ten': 66, 'theh': 67, 'the_peace': 68, 'three': 69, 'thursday': 70, 'tuesday': 71, 'tv': 72, 'two': 73, 'university'

## Collect all the features in one array called "all_features" with thier labels in "all_labels"

In [4]:
all_features = []
all_labels = []

for class_name in os.listdir(os.path.join('arabic_data')):
    print(f"-------------------------------- {class_name} -----------------------------")
    for numpy_file in os.listdir(os.path.join('arabic_data', class_name)):
        if numpy_file.split(".")[-1] == "npy":
            # print(numpy_file)
            # load the numpy file 
            numpy_file_path = os.path.join('arabic_data', class_name, numpy_file)
            numpy_array = np.load(numpy_file_path)
            all_features.append(numpy_array)
            all_labels.append(class_to_index[class_name])

## Save all_features and all_labels lists to external file called "english_data.pickle"

In [170]:
# save all_landmarks and labels in file called data.pickle using pickle module 
data_file = open('arabic_data.pickle','wb')
pickle.dump({'all_features':all_features, 'all_labels':all_labels}, data_file)
data_file.close()

## Load all_features and all_labels to use them

In [2]:
# load the values from the file 
data_file = open('arabic_data.pickle', 'rb')
data = pickle.load(data_file)

all_features = data['all_features']
all_labels = data['all_labels']

data_file.close()

## Convert all_features and all_labels to arrays

In [3]:
# convert the data to arrays 
all_features_array = np.array(all_features)
all_labels_array = np.array(all_labels)

print(type(all_features_array))
print(type(all_labels_array))

print(all_features_array.shape)
print(all_labels_array.shape)

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
(180000, 84)
(180000,)


# 4- Dataset Splitting

In [4]:
from sklearn.model_selection import train_test_split

In [6]:
x_train, x_test, y_train, y_test = train_test_split(all_features_array, all_labels_array, test_size=0.2, shuffle=True, stratify=all_labels)

# 5- Model Building 

In [7]:
from sklearn.ensemble import RandomForestClassifier

In [8]:
model_random_forest = RandomForestClassifier()

In [9]:
model_random_forest.fit(x_train, y_train)

RandomForestClassifier()

## Save the model 

In [13]:
# save the model
model_file = open('arabic_model.pickle','wb')
pickle.dump({'model':model_random_forest}, model_file)
model_file.close()

In [13]:
# load model
model_file = open('arabic_model.pickle','rb')
model_dict = pickle.load(model_file)
model = model_dict['model']

# 6-Model Evaluation 

In [15]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, accuracy_score, precision_score, recall_score, f1_score, classification_report

## Calculate y_predict

In [5]:
# get the y_predict 
y_predict = model.predict(x_test)
print(y_predict)

## Calculate confusion matrix

In [ ]:
cm = confusion_matrix(y_test, y_predict)
for i in cm:
    print(i)

## Calculate Accuracy

In [18]:
accuracy = accuracy_score(y_test, y_predict)
print(accuracy)

0.9988888888888889


## Calculate Precision

In [19]:
precision = precision_score(y_test, y_predict, average='micro')
print(precision)

0.9988888888888889


## Calculate F1-Score

In [20]:
f1 = f1_score(y_test, y_predict, average='micro')
print(f1)

0.9988888888888889


## Calculate Recall

In [21]:
recall = recall_score(y_test, y_predict, average='micro')
print(recall)

0.9988888888888889


## Classification Report

In [22]:
report = classification_report(y_test, y_predict)
print(report)

              precision    recall  f1-score   support

           0       1.00      1.00      1.00       400
           1       1.00      1.00      1.00       400
           2       1.00      1.00      1.00       400
           3       1.00      0.99      0.99       400
           4       1.00      1.00      1.00       400
           5       1.00      0.98      0.99       400
           6       1.00      1.00      1.00       400
           7       1.00      1.00      1.00       400
           8       1.00      1.00      1.00       400
           9       1.00      1.00      1.00       400
          10       1.00      1.00      1.00       400
          11       1.00      1.00      1.00       400
          12       1.00      1.00      1.00       400
          13       1.00      1.00      1.00       400
          14       1.00      1.00      1.00       400
          15       1.00      1.00      1.00       400
          16       1.00      0.99      1.00       400
          17       1.00    

# 7- Test model on videos

## Load google model

In [38]:
# create object just focus on the hands 
mp_hands = mp.solutions.hands

# get the model that detect hand_landmarks 
hands = mp_hands.Hands(static_image_mode=True, min_detection_confidence=0.5)

## Load my model model 

In [39]:
# load model
model_file = open('arabic_model.pickle','rb')
model_dict = pickle.load(model_file)
model = model_dict['model']
model_file.close()

## Create label map

In [40]:
classes = []
for class_name in os.listdir(os.path.join('arabic_data')):
    if len(os.listdir(os.path.join('arabic_data', class_name))) != 0:
        classes.append(class_name)
print(classes)

['aeen', 'alef', 'allah', 'bad', 'bank', 'bathroom', 'beh', 'cairo', 'chair', 'child', 'college', 'daaad', 'daal', 'deaf', 'dish', 'divorce', 'egypt', 'eight', 'engagement', 'engineer', 'father', 'feh', 'five', 'four', 'geem', 'gheen', 'girl_or_daughter', 'green', 'haa', 'happy', 'hehh', 'help', 'home_or_house', 'hospital', 'hungry', 'ismaalia', 'i_love_you', 'i_or_me', 'kaaf', 'khaa', 'laam', 'luxor', 'meem', 'monday', 'nine', 'noon', 'one', 'orange', 'pray', 'qaaf', 'raa', 'saaad', 'saturday', 'school', 'seen', 'seven', 'sharkia', 'sheen', 'sign', 'six', 'sleep_n_v', 'son_or_boy', 'tah', 'teacher', 'teen_fruit', 'teh', 'ten', 'theh', 'the_peace', 'three', 'thursday', 'tuesday', 'tv', 'two', 'university', 'waow', 'water', 'wednesday', 'what', 'where', 'white', 'work_v', 'yeeh', 'yellow', 'yes', 'you', 'your_name', 'zaal', 'zah', 'zeen']


In [41]:
index_to_class = {index:class_name for index, class_name in enumerate(classes)}
print(index_to_class)

{0: 'aeen', 1: 'alef', 2: 'allah', 3: 'bad', 4: 'bank', 5: 'bathroom', 6: 'beh', 7: 'cairo', 8: 'chair', 9: 'child', 10: 'college', 11: 'daaad', 12: 'daal', 13: 'deaf', 14: 'dish', 15: 'divorce', 16: 'egypt', 17: 'eight', 18: 'engagement', 19: 'engineer', 20: 'father', 21: 'feh', 22: 'five', 23: 'four', 24: 'geem', 25: 'gheen', 26: 'girl_or_daughter', 27: 'green', 28: 'haa', 29: 'happy', 30: 'hehh', 31: 'help', 32: 'home_or_house', 33: 'hospital', 34: 'hungry', 35: 'ismaalia', 36: 'i_love_you', 37: 'i_or_me', 38: 'kaaf', 39: 'khaa', 40: 'laam', 41: 'luxor', 42: 'meem', 43: 'monday', 44: 'nine', 45: 'noon', 46: 'one', 47: 'orange', 48: 'pray', 49: 'qaaf', 50: 'raa', 51: 'saaad', 52: 'saturday', 53: 'school', 54: 'seen', 55: 'seven', 56: 'sharkia', 57: 'sheen', 58: 'sign', 59: 'six', 60: 'sleep_n_v', 61: 'son_or_boy', 62: 'tah', 63: 'teacher', 64: 'teen_fruit', 65: 'teh', 66: 'ten', 67: 'theh', 68: 'the_peace', 69: 'three', 70: 'thursday', 71: 'tuesday', 72: 'tv', 73: 'two', 74: 'univers

## Convert english text to arabic text

In [62]:
english_to_arabic = {'aeen':display_arabic_text('ع'), 'alef':display_arabic_text('أ'), 'allah':display_arabic_text('الله'),
                    'bad':display_arabic_text('سىء'), 'bank':display_arabic_text('بنك'), 'bathroom':display_arabic_text('حمام'),
                    'beh':display_arabic_text('ب'), 'cairo':display_arabic_text('القاهرة'), 'chair':display_arabic_text('كرسي'),
                    'child':display_arabic_text('طفل'), 'college':display_arabic_text('كلية'), 'daaad':display_arabic_text('ض'),
                    'daal':display_arabic_text('د'), 'deaf':display_arabic_text('أصم'), 'dish':display_arabic_text('طبق'),
                    'divorce':display_arabic_text('طلاق/مطلق/مطلقة'), 'egypt':display_arabic_text('مصر'), 'eight':display_arabic_text('ثمانية'),
                    'engagement':display_arabic_text('خطوبة'), 'engineer':display_arabic_text('مهندس'), 'father':display_arabic_text('أب'),
                    'feh':display_arabic_text('ف'), 'five':display_arabic_text('خمسة'), 'four':display_arabic_text('أربعة'),
                    'geem':display_arabic_text('ج'), 'gheen':display_arabic_text('غ'), 'girl_or_daughter':display_arabic_text('بنت/أبنة'),
                    'green':display_arabic_text('أخضر'), 'haa':display_arabic_text('ح'), 'happy':display_arabic_text('سعيد'),
                    'hehh':display_arabic_text('ه'), 'help':display_arabic_text('مساعدة'), 'home_or_house':display_arabic_text('منزل/بيت'),
                    'hospital':display_arabic_text('مستشفى'), 'hungry':display_arabic_text('جائع'), 'ismaalia':display_arabic_text('الاسماعيلية'),
                    'i_love_you':display_arabic_text('أحبك'), 'i_or_me':display_arabic_text('انا'), 'kaaf':display_arabic_text('ك'),
                    'khaa':display_arabic_text('خ'), 'laam':display_arabic_text('ل'), 'luxor':display_arabic_text('الأقصر'),
                    'meem':display_arabic_text('م'), 'monday':display_arabic_text('الأثنين'), 'nine':display_arabic_text('تسعة'),
                    'noon':display_arabic_text('ن'), 'one':display_arabic_text('واحد'), 'orange':display_arabic_text('برتقال'),
                    'pray':display_arabic_text('الصلاة'), 'qaaf':display_arabic_text('ق'), 'raa':display_arabic_text('ر'),
                    'saaad':display_arabic_text('ص'), 'saturday':display_arabic_text('السبت'), 'school':display_arabic_text('مدرسة'),
                    'seen':display_arabic_text('س'), 'seven':display_arabic_text('سبعة'), 'sharkia':display_arabic_text('الشرقية'),
                    'sheen':display_arabic_text('ش'), 'sign':display_arabic_text('اشارة'), 'six':display_arabic_text('ستة'),
                    'sleep_n_v':display_arabic_text('ينام/نوم'), 'son_or_boy':display_arabic_text('أبن/ولد'), 'tah':display_arabic_text('ط'),
                    'teacher':display_arabic_text('أستاذ/معلم'), 'teen_fruit':display_arabic_text('تين'), 'teh':display_arabic_text('ت'),
                    'ten':display_arabic_text('عشرة'), 'theh':display_arabic_text('ث'), 'the_peace':display_arabic_text('السلام'),
                    'three':display_arabic_text('ثلاثة'), 'thursday':display_arabic_text('الخميس'), 'tuesday':display_arabic_text('الثلاثاء'),
                    'tv':display_arabic_text('التيليفزيون'), 'two':display_arabic_text('أثنين'), 'university':display_arabic_text('الجامعة'),
                    'waow':display_arabic_text('و'), 'water':display_arabic_text('ماء'), 'wednesday':display_arabic_text('الأربعاء'),
                    'what':display_arabic_text('أيه/ماذا'), 'where':display_arabic_text('فين/أين'), 'white':display_arabic_text('أبيض'),
                    'work_v':display_arabic_text('يعمل/يشتغل'), 'yeeh':display_arabic_text('ي'), 'yellow':display_arabic_text('أصفر'),
                    'yes':display_arabic_text('نعم'), 'you':display_arabic_text('أنت'), 'your_name':display_arabic_text('أسمك'),
                    'zaal':display_arabic_text('ذ'), 'zah':display_arabic_text('ظ'), 'zeen':display_arabic_text('ز')}

In [63]:
new_english_to_arabic = {'aeen':'ع', 'alef':'أ', 'allah':'الله',
                    'bad':'سىء', 'bank':'بنك', 'bathroom':'حمام',
                    'beh':'ب', 'cairo':'القاهرة', 'chair':'كرسي',
                    'child':'طفل', 'college':'كلية', 'daaad':'ض',
                    'daal':'د', 'deaf':'أصم', 'dish':'طبق',
                    'divorce':'طلاق/مطلق/مطلقة', 'egypt':'مصر', 'eight':'ثمانية',
                    'engagement':'خطوبة', 'engineer':'مهندس', 'father':'أب',
                    'feh':'ف', 'five':'خمسة', 'four':'أربعة',
                    'geem':'ج', 'gheen':'غ', 'girl_or_daughter':'بنت/أبنة',
                    'green':'أخضر', 'haa':'ح', 'happy':'سعيد',
                    'hehh':'ه', 'help':'مساعدة', 'home_or_house':'منزل/بيت',
                    'hospital':'مستشفى', 'hungry':'جائع', 'ismaalia':'الاسماعيلية',
                    'i_love_you':'أحبك', 'i_or_me':'انا', 'kaaf':'ك',
                    'khaa':'خ', 'laam':'ل', 'luxor':'الأقصر',
                    'meem':'م', 'monday':'الأثنين', 'nine':'تسعة',
                    'noon':'ن', 'one':'واحد', 'orange':'برتقال',
                    'pray':'الصلاة', 'qaaf':'ق', 'raa':'ر',
                    'saaad':'ص', 'saturday':'السبت', 'school':'مدرسة',
                    'seen':'س', 'seven':'سبعة', 'sharkia':'الشرقية',
                    'sheen':'ش', 'sign':'اشارة', 'six':'ستة',
                    'sleep_n_v':'ينام/نوم', 'son_or_boy':'أبن/ولد', 'tah':'ط',
                    'teacher':'أستاذ/معلم', 'teen_fruit':'تين', 'teh':'ت',
                    'ten':'عشرة', 'theh':'ث', 'the_peace':'السلام',
                    'three':'ثلاثة', 'thursday':'الخميس', 'tuesday':'الثلاثاء',
                    'tv':'التيليفزيون', 'two':'أثنين', 'university':'الجامعة',
                    'waow':'و', 'water':'ماء', 'wednesday':'الأربعاء',
                    'what':'أيه/ماذا', 'where':'فين/أين', 'white':'أبيض',
                    'work_v':'يعمل/يشتغل', 'yeeh':'ي', 'yellow':'أصفر',
                    'yes':'نعم', 'you':'أنت', 'your_name':'أسمك',
                    'zaal':'ذ', 'zah':'ظ', 'zeen':'ز'}

## Function: take image and output label and its probability

In [56]:
# define method to predict image with the random forest model
# image -> label, probablity of it 

def model_image_predict(image):
    
    # convert image to rgb 
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    # get landmarks
    result = hands.process(image_rgb)
    
    # define list to put all the landmarks in it 
    all_features = []
    
    # define list to put in it x,y values for each landmarks 
    current_image_landmarks = []

    # get x and y value for each landmark

    # check if there is any detection of hands or not 
    if result.multi_hand_landmarks:

        for hand_landmark in result.multi_hand_landmarks:
            for landmark in hand_landmark.landmark:
                current_image_landmarks.append(landmark.x)
                current_image_landmarks.append(landmark.y)

        # check that the number of landmarks are equal for each image
        if len(current_image_landmarks) < 84:
            current_image_landmarks = current_image_landmarks + [0]*(84-len(current_image_landmarks))

        # append the value of current_image_data in the all_data list
        all_features.append(current_image_landmarks)
        
        # convert the all_landmarks from list to 2d array
        all_features_array = np.array(all_features)
        
        
        prediction = model.predict(all_features_array)
        prediction_with_probability = model.predict_proba(all_features_array)

        return {'class':prediction[0], 'probability':prediction_with_probability[0][prediction[0]]}

## Predict on image

In [57]:
image_path = os.path.join("arabic_data", "aeen", "new_left_3een_frame_1.jpg")
image = cv2.imread(image_path)
prediction = model_image_predict(image)
print(prediction)
print(type(prediction))

{'class': 0, 'probability': 1.0}
<class 'dict'>


## Function: Convert the video to text
Be attention for 2 important things :
- Frame Per Second [FPS]
- Video Status: 
    - 1 sign language 
    - Multiple sign language 

In [65]:
def video_to_text_prediction(video_path):

    # read the video
    video = cv2.VideoCapture(video_path)

    # calculate the frame per second of the video -> take 1 frame per 1/2 second
    fps = round(round(video.get(cv2.CAP_PROP_FPS))/2) # 30 frmas per sec [in each second we will get 2 frames for detection process]

    # define frame_counter variable --> when it reach 15 we will take this frame [so we take a frame after 1/2 second]
    frame_counter = 0

    # get all predictions from the video in all_prediction dictionary 
    all_predictions = dict()
    prediction_id = 0

    status = True  
    while status:
        # read frames from the video
        status, frame = video.read()

        if status == True:
            # count the current frame
            frame_counter += 1
            # here is the frame that will be used for prediction
            if frame_counter % fps == 0:
                prediction = model_image_predict(frame)
                prediction_id += 1 
                all_predictions[prediction_id] = prediction
    
    
    threshold = 0.25
    all_classes = str()
    for i in all_predictions:
        # check if there is a prediction or not
        if all_predictions[i]:
            if all_predictions[i]['probability'] > threshold:
                arabic_text = new_english_to_arabic[index_to_class[all_predictions[i]['class']]]
                all_classes += arabic_text + ' '
    # make unique classes [Cancel repetition]
    previous_class = None
    final_classes = []

    for i in all_classes.rstrip().split():

        if i != previous_class:
            final_classes.append(i)
            previous_class = i

    return " ".join(final_classes)

## Predict on videos

In [66]:
print(video_to_text_prediction(os.path.join("arabic_test_videos", "واحد_أثنين_ثلاثة.mp4")))

واحد أثنين ثلاثة


# 8- Video to text model Deployment

## Endpoint video to text

In [67]:
import mediapipe as mp
import cv2
import numpy as np
import pickle
import os
from flask import Flask, request, jsonify
from celery import Celery

In [7]:
app = Flask(__name__)

@app.route("/video_to_text_arabic", methods=['POST'])
def video_to_text():

    # upload file
    # this is the video itself
    video = request.files['video']
    file_name = video.filename.split("/")[-1]
    
    # make a fixed video name to make overrite and make sure that the uploaded file just have only 1 video file
    file_name = "new_video.mp4"
    
    print("this is the path of the video", file_name)

    # save the video -> to access it and make the detection process
    # after saving the video we will get the 1st parameter for the detection method -> [video_path]
    
   # save the new video
    video.save(os.path.join('uploaded', file_name))
        
        

    # apply model on it
    output = video_to_text_prediction(os.path.join('uploaded', file_name))
    
    return jsonify([{"text":output}])


app.run(port=5000)

# 9- Test in real time

In [25]:
import mediapipe as mp
import cv2
import numpy as np
import pickle
import os

# load model 
model_file = open('arabic_model.pickle','rb')
model_dict = pickle.load(model_file)
model = model_dict['model']
model_file.close()

In [26]:
classes = []
for class_name in os.listdir(os.path.join('arabic_data')):
    if len(os.listdir(os.path.join('arabic_data', class_name))) != 0:
        classes.append(class_name)
print(classes)

['aeen', 'alef', 'allah', 'bad', 'bank', 'bathroom', 'beh', 'cairo', 'chair', 'child', 'college', 'daaad', 'daal', 'deaf', 'dish', 'divorce', 'egypt', 'eight', 'engagement', 'engineer', 'father', 'feh', 'five', 'four', 'geem', 'gheen', 'girl_or_daughter', 'green', 'haa', 'happy', 'hehh', 'help', 'home_or_house', 'hospital', 'hungry', 'ismaalia', 'i_love_you', 'i_or_me', 'kaaf', 'khaa', 'laam', 'luxor', 'meem', 'monday', 'nine', 'noon', 'one', 'orange', 'pray', 'qaaf', 'raa', 'saaad', 'saturday', 'school', 'seen', 'seven', 'sharkia', 'sheen', 'sign', 'six', 'sleep_n_v', 'son_or_boy', 'tah', 'teacher', 'teen_fruit', 'teh', 'ten', 'theh', 'the_peace', 'three', 'thursday', 'tuesday', 'tv', 'two', 'university', 'waow', 'water', 'wednesday', 'what', 'where', 'white', 'work_v', 'yeeh', 'yellow', 'yes', 'you', 'your_name', 'zaal', 'zah', 'zeen']


In [27]:
index_to_class = {index:class_name for index, class_name in enumerate(classes)}
print(index_to_class)

{0: 'aeen', 1: 'alef', 2: 'allah', 3: 'bad', 4: 'bank', 5: 'bathroom', 6: 'beh', 7: 'cairo', 8: 'chair', 9: 'child', 10: 'college', 11: 'daaad', 12: 'daal', 13: 'deaf', 14: 'dish', 15: 'divorce', 16: 'egypt', 17: 'eight', 18: 'engagement', 19: 'engineer', 20: 'father', 21: 'feh', 22: 'five', 23: 'four', 24: 'geem', 25: 'gheen', 26: 'girl_or_daughter', 27: 'green', 28: 'haa', 29: 'happy', 30: 'hehh', 31: 'help', 32: 'home_or_house', 33: 'hospital', 34: 'hungry', 35: 'ismaalia', 36: 'i_love_you', 37: 'i_or_me', 38: 'kaaf', 39: 'khaa', 40: 'laam', 41: 'luxor', 42: 'meem', 43: 'monday', 44: 'nine', 45: 'noon', 46: 'one', 47: 'orange', 48: 'pray', 49: 'qaaf', 50: 'raa', 51: 'saaad', 52: 'saturday', 53: 'school', 54: 'seen', 55: 'seven', 56: 'sharkia', 57: 'sheen', 58: 'sign', 59: 'six', 60: 'sleep_n_v', 61: 'son_or_boy', 62: 'tah', 63: 'teacher', 64: 'teen_fruit', 65: 'teh', 66: 'ten', 67: 'theh', 68: 'the_peace', 69: 'three', 70: 'thursday', 71: 'tuesday', 72: 'tv', 73: 'two', 74: 'univers

## Convert english text to arabic text

In [35]:
english_to_arabic = {'aeen':display_arabic_text('ع'), 'alef':display_arabic_text('أ'), 'allah':display_arabic_text('الله'),
                    'bad':display_arabic_text('سىء'), 'bank':display_arabic_text('بنك'), 'bathroom':display_arabic_text('حمام'),
                    'beh':display_arabic_text('ب'), 'cairo':display_arabic_text('القاهرة'), 'chair':display_arabic_text('كرسي'),
                    'child':display_arabic_text('طفل'), 'college':display_arabic_text('كلية'), 'daaad':display_arabic_text('ض'),
                    'daal':display_arabic_text('د'), 'deaf':display_arabic_text('أصم'), 'dish':display_arabic_text('طبق'),
                    'divorce':display_arabic_text('طلاق/مطلق/مطلقة'), 'egypt':display_arabic_text('مصر'), 'eight':display_arabic_text('ثمانية'),
                    'engagement':display_arabic_text('خطوبة'), 'engineer':display_arabic_text('مهندس'), 'father':display_arabic_text('أب'),
                    'feh':display_arabic_text('ف'), 'five':display_arabic_text('خمسة'), 'four':display_arabic_text('أربعة'),
                    'geem':display_arabic_text('ج'), 'gheen':display_arabic_text('غ'), 'girl_or_daughter':display_arabic_text('بنت/أبنة'),
                    'green':display_arabic_text('أخضر'), 'haa':display_arabic_text('ح'), 'happy':display_arabic_text('سعيد'),
                    'hehh':display_arabic_text('ه'), 'help':display_arabic_text('مساعدة'), 'home_or_house':display_arabic_text('منزل/بيت'),
                    'hospital':display_arabic_text('مستشفى'), 'hungry':display_arabic_text('جائع'), 'ismaalia':display_arabic_text('الاسماعيلية'),
                    'i_love_you':display_arabic_text('أحبك'), 'i_or_me':display_arabic_text('انا'), 'kaaf':display_arabic_text('ك'),
                    'khaa':display_arabic_text('خ'), 'laam':display_arabic_text('ل'), 'luxor':display_arabic_text('الأقصر'),
                    'meem':display_arabic_text('م'), 'monday':display_arabic_text('الأثنين'), 'nine':display_arabic_text('تسعة'),
                    'noon':display_arabic_text('ن'), 'one':display_arabic_text('واحد'), 'orange':display_arabic_text('برتقال'),
                    'pray':display_arabic_text('الصلاة'), 'qaaf':display_arabic_text('ق'), 'raa':display_arabic_text('ر'),
                    'saaad':display_arabic_text('ص'), 'saturday':display_arabic_text('السبت'), 'school':display_arabic_text('مدرسة'),
                    'seen':display_arabic_text('س'), 'seven':display_arabic_text('سبعة'), 'sharkia':display_arabic_text('الشرقية'),
                    'sheen':display_arabic_text('ش'), 'sign':display_arabic_text('اشارة'), 'six':display_arabic_text('ستة'),
                    'sleep_n_v':display_arabic_text('ينام/نوم'), 'son_or_boy':display_arabic_text('أبن/ولد'), 'tah':display_arabic_text('ط'),
                    'teacher':display_arabic_text('أستاذ/معلم'), 'teen_fruit':display_arabic_text('تين'), 'teh':display_arabic_text('ت'),
                    'ten':display_arabic_text('عشرة'), 'theh':display_arabic_text('ث'), 'the_peace':display_arabic_text('السلام'),
                    'three':display_arabic_text('ثلاثة'), 'thursday':display_arabic_text('الخميس'), 'tuesday':display_arabic_text('الثلاثاء'),
                    'tv':display_arabic_text('التيليفزيون'), 'two':display_arabic_text('أثنين'), 'university':display_arabic_text('الجامعة'),
                    'waow':display_arabic_text('و'), 'water':display_arabic_text('ماء'), 'wednesday':display_arabic_text('الأربعاء'),
                    'what':display_arabic_text('أيه/ماذا'), 'where':display_arabic_text('فين/أين'), 'white':display_arabic_text('أبيض'),
                    'work_v':display_arabic_text('يعمل/يشتغل'), 'yeeh':display_arabic_text('ي'), 'yellow':display_arabic_text('أصفر'),
                    'yes':display_arabic_text('نعم'), 'you':display_arabic_text('أنت'), 'your_name':display_arabic_text('أسمك'),
                    'zaal':display_arabic_text('ذ'), 'zah':display_arabic_text('ظ'), 'zeen':display_arabic_text('ز')}

In [83]:
new_english_to_arabic = {'aeen':'ع', 'alef':'أ', 'allah':'الله',
                    'bad':'سىء', 'bank':'بنك', 'bathroom':'حمام',
                    'beh':'ب', 'cairo':'القاهرة', 'chair':'كرسي',
                    'child':'طفل', 'college':'كلية', 'daaad':'ض',
                    'daal':'د', 'deaf':'أصم', 'dish':'طبق',
                    'divorce':'طلاق/مطلق/مطلقة', 'egypt':'مصر', 'eight':'ثمانية',
                    'engagement':'خطوبة', 'engineer':'مهندس', 'father':'أب',
                    'feh':'ف', 'five':'خمسة', 'four':'أربعة',
                    'geem':'ج', 'gheen':'غ', 'girl_or_daughter':'بنت/أبنة',
                    'green':'أخضر', 'haa':'ح', 'happy':'سعيد',
                    'hehh':'ه', 'help':'مساعدة', 'home_or_house':'منزل/بيت',
                    'hospital':'مستشفى', 'hungry':'جائع', 'ismaalia':'الاسماعيلية',
                    'i_love_you':'أحبك', 'i_or_me':'انا', 'kaaf':'ك',
                    'khaa':'خ', 'laam':'ل', 'luxor':'الأقصر',
                    'meem':'م', 'monday':'الأثنين', 'nine':'تسعة',
                    'noon':'ن', 'one':'واحد', 'orange':'برتقال',
                    'pray':'الصلاة', 'qaaf':'ق', 'raa':'ر',
                    'saaad':'ص', 'saturday':'السبت', 'school':'مدرسة',
                    'seen':'س', 'seven':'سبعة', 'sharkia':'الشرقية',
                    'sheen':'ش', 'sign':'اشارة', 'six':'ستة',
                    'sleep_n_v':'ينام/نوم', 'son_or_boy':'أبن/ولد', 'tah':'ط',
                    'teacher':'أستاذ/معلم', 'teen_fruit':'تين', 'teh':'ت',
                    'ten':'عشرة', 'theh':'ث', 'the_peace':'السلام',
                    'three':'ثلاثة', 'thursday':'الخميس', 'tuesday':'الثلاثاء',
                    'tv':'التيليفزيون', 'two':'أثنين', 'university':'الجامعة',
                    'waow':'و', 'water':'ماء', 'wednesday':'الأربعاء',
                    'what':'أيه/ماذا', 'where':'فين/أين', 'white':'أبيض',
                    'work_v':'يعمل/يشتغل', 'yeeh':'ي', 'yellow':'أصفر',
                    'yes':'نعم', 'you':'أنت', 'your_name':'أسمك',
                    'zaal':'ذ', 'zah':'ظ', 'zeen':'ز'}

print(new_english_to_arabic)

{'aeen': 'ع', 'alef': 'أ', 'allah': 'الله', 'bad': 'سىء', 'bank': 'بنك', 'bathroom': 'حمام', 'beh': 'ب', 'cairo': 'القاهرة', 'chair': 'كرسي', 'child': 'طفل', 'college': 'كلية', 'daaad': 'ض', 'daal': 'د', 'deaf': 'أصم', 'dish': 'طبق', 'divorce': 'طلاق/مطلق/مطلقة', 'egypt': 'مصر', 'eight': 'ثمانية', 'engagement': 'خطوبة', 'engineer': 'مهندس', 'father': 'أب', 'feh': 'ف', 'five': 'خمسة', 'four': 'أربعة', 'geem': 'ج', 'gheen': 'غ', 'girl_or_daughter': 'بنت/أبنة', 'green': 'أخضر', 'haa': 'ح', 'happy': 'سعيد', 'hehh': 'ه', 'help': 'مساعدة', 'home_or_house': 'منزل/بيت', 'hospital': 'مستشفى', 'hungry': 'جائع', 'ismaalia': 'الاسماعيلية', 'i_love_you': 'أحبك', 'i_or_me': 'انا', 'kaaf': 'ك', 'khaa': 'خ', 'laam': 'ل', 'luxor': 'الأقصر', 'meem': 'م', 'monday': 'الأثنين', 'nine': 'تسعة', 'noon': 'ن', 'one': 'واحد', 'orange': 'برتقال', 'pray': 'الصلاة', 'qaaf': 'ق', 'raa': 'ر', 'saaad': 'ص', 'saturday': 'السبت', 'school': 'مدرسة', 'seen': 'س', 'seven': 'سبعة', 'sharkia': 'الشرقية', 'sheen': 'ش', 'sign

## Process

In [30]:
# create objects just focus on the hands 
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles

# get the model that detect hand_landmarks 
hands = mp_hands.Hands(static_image_mode=True, min_detection_confidence=0.5)

# setup webcam 
camera = cv2.VideoCapture(0)

# # define empty string 
# statement = str()

# loop to read frames from the webcam
while camera.isOpened():

    # read frames from the webcam
    status, frame = camera.read()
    
    # flip the frame 
    frame = cv2.flip(frame, 1)

    # convert frame to rgb
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # get hands detection on the rgb image 
    result = hands.process(frame_rgb)
    
    # define lists
    current_image_landmarks = []
    all_features = []
    
    if result.multi_hand_landmarks:
        for hand_landmark in result.multi_hand_landmarks:
            # draw the landmarks
            mp_drawing.draw_landmarks(
                frame,
                hand_landmark, 
                mp_hands.HAND_CONNECTIONS,
                mp_drawing_styles.get_default_hand_landmarks_style(),
                mp_drawing_styles.get_default_hand_connections_style())
                
            for landmark in hand_landmark.landmark:
                current_image_landmarks.append(landmark.x)
                current_image_landmarks.append(landmark.y)

        # check that the number of landmarks are equal for each image
        if len(current_image_landmarks) < 84:
            current_image_landmarks = current_image_landmarks + [0]*(84-len(current_image_landmarks))

        # append the value of current_image_data in the all_data list
        all_features.append(current_image_landmarks)
        
        # convert the all_landmarks from list to 2d array
        all_features_array = np.array(all_features)
        
        prediction = model.predict(all_features_array)
        prediction_with_probability = model.predict_proba(all_features_array)
        
        class_label = index_to_class[prediction[0]]
        class_label = english_to_arabic[class_label]
        full_text = f'{class_label}, {str(prediction_with_probability[0][prediction[0]] * 100)[:4]} %'
        
        cv2.putText(frame, full_text, (100,100), cv2.FONT_HERSHEY_SIMPLEX, 1.3, (0,0,255), 3, cv2.LINE_AA)        
    
    cv2.imshow('Window', frame)
    if cv2.waitKey(1) &  0xFF == ord('q'):
        break

camera.release()
cv2.destroyAllWindows()

In [43]:
camera.release()
cv2.destroyAllWindows()

## 10- Covert text to video

In [177]:
import cv2
import os 
import string 

In [178]:
# list of exist videos called mapping videos
mapping_video_list = []
mapping_videos = os.listdir(os.path.join("mapping_videos_arabic"))
for video in mapping_videos:
    mapping_video_list.append(video.split(".")[0])

print(mapping_video_list)

['0', '1', '10', '2', '3', '4', '5', '6', '7', '8', '9', 'ء', 'آ', 'أ', 'أب', 'أبن', 'أبيض', 'أثنبن', 'أحب', 'أحبك', 'أحمر', 'أخضر', 'أخوات', 'أربعة', 'أربعه', 'أرمل', 'أرملة', 'أرمله', 'أزرق', 'أسبوع', 'أسمك', 'أسمي', 'أشارة', 'أشاره', 'أصفر', 'أصم', 'أم', 'أنا', 'أنت', 'أين', 'أيه', 'ؤ', 'إ', 'إشارة', 'إشاره', 'ا', 'اب', 'ابن', 'ابيض', 'اثنين', 'احب', 'احبك', 'احمر', 'اخضر', 'اخوات', 'اربعة', 'اربعه', 'ارمل', 'ارملة', 'ارمله', 'ازرق', 'اسبوع', 'اسمك', 'اسمي', 'اشارة', 'اشاره', 'اصفر', 'اصم', 'الأبيض', 'الأثنين', 'الأحد', 'الأحمر', 'الأخضر', 'الأخوات', 'الأربعاء', 'الأزرق', 'الأسماعيلية', 'الأسماعيليه', 'الأصفر', 'الأقصر', 'الإسماعيلية', 'الإسماعيليه', 'الابيض', 'الاثنين', 'الاحد', 'الاحمر', 'الاخضر', 'الاخوات', 'الاربعاء', 'الازرق', 'الاسماعيلية', 'الاسماعيليه', 'الاصفر', 'الاقصر', 'البحر', 'البحرالأحمر', 'البحرالاحمر', 'البيت', 'الترابيزة', 'الترابيزه', 'التعارف', 'التعليم', 'التيليفيزيون', 'التين', 'الثلاثاء', 'الجمعة', 'الجمعه', 'الحمام', 'الحمد', 'الحمدلله', 'الخميس', 'الرحمة', '

In [192]:
def text_to_video_prediction_arabic(text:str):
    
    # convert all text to lowercase
    text = text.lower()
    
    # delete any punctuation in the text
    
    # Create a translation table for the translate function
    translator = str.maketrans('', '', '''!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~؟!#$"،,.ٍِـ،/:"><؛×÷‘ًًٌَُ$#@!%^&*)(''') 
    
    # Remove punctuation 
    text = text.translate(translator)  
    
    print(text)
    # split the whole text into separated words
    words_list = text.split(" ")
    
    # define codecc
    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    
    # define the save path 
    output_video_path = os.path.join("created_videos", f"new_created_video.avi")
    
    # define video writer object to concatenate all videos in it -> by default none may be there is no words in the mapping video list
    video_writer = None
    
    for word in words_list:
        if word in mapping_video_list:
            # means there is a video for this words 
            # access this video path 
            video_path = os.path.join("mapping_videos_arabic", f"{word}.mp4")
            
            # read the video
            video = cv2.VideoCapture(video_path)
            
            # extract important video features
            width = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
            height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))
            fps = video.get(cv2.CAP_PROP_FPS)
            
            if video_writer is None:
                video_writer = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))
            
            status = True
            while status:
                
                # read Frames
                status, frame = video.read()
                
                if status == True:
                    # insert the frames of the video in the new created video [frame by frame]
                    video_writer.write(frame)
                
            
            # we get out of the first video and we will cloase it safely now
            video.release()
        
        # has no corresponding words but we can express it by letters
        else:
            
            # split the word to its letter and express the letters by their videos
            word = " ".join(word)
            
            for letter in word:
                if letter == "ة":
                    video_path = os.path.join("mapping_videos_arabic", "ه.mp4")
                else:
                    video_path = os.path.join("mapping_videos_arabic", f"{letter}.mp4")
            
                # read the video
                video = cv2.VideoCapture(video_path)

                # extract important video features
                width = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
                height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))
                fps = video.get(cv2.CAP_PROP_FPS)

                if video_writer is None:
                    video_writer = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))

                status = True
                while status:

                    # read Frames
                    status, frame = video.read()

                    if status == True:
                        # insert the frames of the video in the new created video [frame by frame]
                        video_writer.write(frame)

                # we get out of the first video and we will cloase it safely now
                video.release()
            
            
    # here we insert all the frames of all videos in the new created video -> we will cloase this video safely 
    if video_writer != None:
        video_writer.release()
        return output_video_path
    

# 11- Test text to video model

In [203]:
print(text_to_video_prediction_arabic("انا اسمي محمد"))

انا اسمي محمد
created_videos\new_created_video.avi


# 12- Text to video model deployment

## Endpoint text to video 

In [200]:
import os 
from flask import Flask, request, send_file

In [8]:
app = Flask(__name__)

@app.route("/text_to_video_arabic", methods=['POST'])
def text_to_video():

    # request the text from the client as form data
    # text_value = request.form.get('text')
    
    # request the text as raw data 
    text_value = str(request.data ,encoding='utf-8')
    
    print(type(text_value))
    print(text_value)
    

    # apply model on the text
    output_video_path = text_to_video_prediction_arabic(text_value)
    print(output_video_path)
    
    return send_file(output_video_path)


app.run(port=5000)